In [1]:
using Flux
include("TwoSpin_env_gpu.jl")


mutable struct agtQ
    in_size::Int
    out_size::Int
    n_dense::Int
    ϵ::Float32
    γ::Float32
    HF_TL::Matrix{Float32}
    K_TL::Matrix{Float32}
    Kp_TL::Matrix{Float32}
end

function init_nQ(en::TS_env, n::Int=32, γ0::Float32=0.9, ϵ0::Float32=1.0)
    #H_0,V_tのパラメータの数＋K_tの行列＋H_F^a(t)の行列
    in_size::Int = en.num_parm + 2*en.HS_size^2 

    #K'(t)の行列を出力
    out_size::Int = en.HS_size^2

    #中間層のニューロンの数
    n_dense::Int = n

    #乱数発生用のパラメータ
    ϵ::Float32 = ϵ0

    #割引率
    γ::Float32 = γ0

    HF_TL = zeros(Float32, en.t_size, en.HS_size^2)
    K_TL = zeros(Float32, en.t_size, en.HS_size^2)
    Kp_TL = zeros(Float32, en.t_size, en.HS_size^2)

    return in_size, out_size, n_dense, ϵ, γ, HF_TL, K_TL, Kp_TL
end

init_nQ (generic function with 4 methods)

In [2]:
using CUDA

In [18]:
function micro_motion(Kp_t::Vector{Float32}, K_t::Vector{Float32}, en::TS_env, t::Int)
    Kp = VtoM(Kp_t,en)
    K_t_new::Vector{Float32} = K_t + (2pi/en.t_size/en.Ω) * Kp_t 
    Kt = VtoM(K_t_new,en)
    HF_m = Hermitian(ComplexF32.(exp(1.0im*Kt)*(en.H_0 + en.V_t*sin(2pi*t/en.t_size) - Kp)*exp(-1.0im*Kt)))
    HF = MtoV(HF_m, en)
    return K_t_new, HF
end
    
    function micro_motion2(Kp_t::Vector{Float32}, K_t::Vector{Float32}, en::TS_env, t::Int)
        Kp = VtoM(Kp_t,en)
        K_t_new::Vector{Float32} = K_t + (2pi/en.t_size/en.Ω) * Kp_t 
        Kt = VtoM(K_t_new,en)
        HF_m = Hermitian(ComplexF32.(exp(1.0im*Kt)*(en.H_0 + en.V_t*sin(2pi*t/en.t_size) - Kp)*exp(-1.0im*Kt)))
        HF = MtoV(HF_m,en)
        return HF
    end
    
    
    function diff_norm(V::Vector{Float32}, en::TS_env)
        M = VtoM(V,en)
        e, v = eigen(M)
        #n::Float32 = V' * V
        n::Float32 = e' * e
        #n = sum(e[n]^2 for n in 1:size(e))
        return n
    end
    
    
    #lossの関数
    function loss_fn(en::TS_env, ag::agtQ, t::Int, sw::Int)
        l::Float32 = 0.0
        for n in 1:(en.t_size-1) 
            if(n<t)
                lt = t-n
            elseif(n==t)
                lt = en.t_size
            else
                lt = t-n+en.t_size
                if(sw==1) 
                    break
                end
            end
            l -= (ag.γ^(n-1)) * diff_norm(ag.HF_TL[t,:]-ag.HF_TL[lt,:],en)
        end
        return l
    end
    
    function loss_fn_given(en::TS_env, ag::agtQ,H_t::Vector{Float32}, t::Int, sw::Int)
        l::Float32 = 0.0
        for n in 1:(en.t_size-1) 
            if(n<t)
                lt = t-n
            elseif(n==t)
                lt = en.t_size
            else
                lt = t-n+en.t_size
                if(sw==1) 
                    break
                end
            end
            l -= (ag.γ^(n-1)) * diff_norm((H_t-ag.HF_TL[lt,:]),en)
        end
        return l
    end
    
    function loss_fn_simple(en::TS_env, HF_given::Vector{Float32}, HF_calc::Vector{Float32})
        l = -diff_norm(HF_given - HF_calc,en)
        return l
    end
    
    function loss_fn_hybrid(en::TS_env, ag::agtQ, HF_given::Vector{Float32}, HF_calc::Vector{Float32}, t::Int)
        l::Float32 = 0.0
        for n in 1:(en.t_size-1) 
            if(n<t)
                lt = t-n
            elseif(n==t)
                lt = en.t_size
            else
                break
                #lt = t-n+en.t_size
            end
            l += ag.ϵ*(ag.γ^(n-1)) * diff_norm((HF_calc-ag.HF_TL[lt,:]),en)/en.t_size
        end
        #l += diff_norm(HF_given - HF_calc,en)/en.t_size
        return l
    end
    
    function loss_calc0(model0, en::TS_env, ag::agtQ, t::Int, HF_given::Vector{Float32})
        if(t==1)
            tt=en.t_size
        else
            tt=t-1
        end
        p = [en.Ω, en.ξ*sin(2pi*t/en.t_size), en.Jz, en.Jx, en.hz]
        x = vcat([p, ag.K_TL[tt,:], ag.HF_TL[tt,:]]...)
        Kp = model0(x)
        
        #ag.K_TL[t,:] += Kp
        HF_calc = micro_motion2(Kp, ag.K_TL[tt,:],en,t)
        l = -loss_fn_simple(en, HF_given, HF_calc)
        #l = Kp' * Kp
        return l 
    end
    
    function loss_calc(model0, en::TS_env, ag::agtQ, t::Int, it::Int)
        if(t==1)
            tt=en.t_size
        else
            tt=t-1
        end
        p = [en.Ω, en.ξ*sin(2pi*t/en.t_size), en.Jz, en.Jx, en.hz]
        x = vcat([p, ag.K_TL[tt,:], ag.HF_TL[tt,:]]...)
        Kp = model0(x)
        
        #ag.K_TL[t,:] += Kp
        HF_t = micro_motion2(Kp, ag.K_TL[tt,:],en,t)
        l = -loss_fn_given(en, ag, HF_t, t, it)
        #l = Kp' * Kp
        return l 
    end
    
    function loss_calc_hyb(model0, en::TS_env, ag::agtQ, HF_given::Vector{Float32})
        l::Float32 = 0.0
        kp_sum = zeros(Float32, en.HS_size^2)
        for t in 1:en.t_size
            if(t==1)
                tt=en.t_size
            else
                tt=t-1
            end
            p = [en.Ω, Float32(en.ξ*sin(2pi*t/en.t_size)), en.Jz, en.Jx, en.hz]
            x = vcat([p, ag.K_TL[tt,:], ag.Kp_TL[tt,:]]...)
            xg = gpu(x)
            Kp = Array{Float32}(model0(xg))
            kp_sum += Kp
            #ag.K_TL[t,:] += Kp
            HF_calc = micro_motion2(Kp, ag.K_TL[tt,:],en,t)
            l += loss_fn_hybrid(en,ag, HF_given, HF_calc,t)
            #l += ag.ϵ^2*diff_norm(kp_sum,en)/en.t_size
            #l += diff_norm(kp_sum,en)/en.t_size
            #l += ag.γ^(5*(en.t_size/2 - abs(en.t_size/2-t))) * diff_norm(ag.K_TL[t,:],en)
            #if(t==t_size)
            #    l += diff_norm(HF_calc-ag.HF_TL[1,:],en)
            #end
        end
        l += diff_norm(kp_sum,en)/en.t_size
        return l 
    end
    
    function loss_calc_hyb!(model0, en::TS_env, ag::agtQ, HF_given::Vector{Float32})
        l::Float32 = 0.0
        kp_sum = zeros(Float32, en.HS_size^2)
        for t in 1:en.t_size
            if(t==1)
                tt=en.t_size
            else
                tt=t-1
            end
            p = [en.Ω, en.ξ*sin(2pi*t/en.t_size), en.Jz, en.Jx, en.hz]
            x = vcat([p, ag.K_TL[tt,:], ag.Kp_TL[tt,:]]...)
            xg = gpu(x)
            ag.Kp_TL[t,:] = Array{Float32}(model0(xg))
            kp_sum += ag.Kp_TL[t,:]
            #ag.K_TL[t,:] += Kp
            ag.K_TL[t,:], ag.HF_TL[t,:] = micro_motion(ag.Kp_TL[t,:], ag.K_TL[tt,:],en,t)
            l += loss_fn_hybrid(en,ag, HF_given, ag.HF_TL[t,:],t)
            #l += diff_norm(kp_sum,en)/en.t_size
            #l += ag.γ^(5*(en.t_size/2 - abs(en.t_size/2-t))) * diff_norm(ag.K_TL[t,:],en)
            #l += ag.γ^(5*(en.t_size - t)) * diff_norm(ag.K_TL[t,:],en)
            #if(t==t_size)
            #    l += diff_norm(ag.HF_TL[t,:]-ag.HF_TL[1,:],en)
            #end
        end
        l += diff_norm(kp_sum,en)
        return l, kp_sum/en.t_size
    end
    
    function loss_calc!(model0, en::TS_env, ag::agtQ, t::Int, HF_given::Vector{Float32})
        if(t==1)
            tt=en.t_size
        else
            tt=t-1
        end
        p = [en.Ω, en.ξ*sin(2pi*t/en.t_size), en.Jz, en.Jx, en.hz]
        x = vcat([p, ag.K_TL[tt,:], ag.HF_TL[tt,:]]...)
        Kp = model0(x)
        
        #ag.K_TL[t,:] += Kp
        ag.K_TL[t,:], ag.HF_TL[t,:] = micro_motion(Kp, ag.K_TL[tt,:],en,t)
        l = -loss_fn_simple(en, HF_given, ag.HF_TL[t,:])
        #l = -loss_fn(en, ag, t, it)
        #l = Kp' * Kp
        return l 
    end
    
    function updata_KpK!(en::TS_env, ag::agtQ, Kp_av::Vector{Float32})
        for t in 1:en.t_size
            ag.Kp_TL[t,:] = ag.Kp_TL[t,:] - Kp_av
            ag.K_TL[t,:] = ag.K_TL[t,:] - t*Kp_av
            ag.HF_TL[t,:] = micro_motion2(ag.Kp_TL[t,:], ag.K_TL[t,:], en, t)
        end
    end
    
    
    function init_HF(en::TS_env)
        jp = en.Jz + en.hz
        jm = en.Jz - en.hz
        VHmHV::Vector{Float32} = 4*en.ξ*[0.0, 0.0, jp, 0.0, jp, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -jm, 0.0, 0.0, -jm, 0.0]
        init = MtoV(en.H_0, en) + VHmHV
        return init
    end
    
    using DataFrames
    using CSV
    using BSON: @save
    using Plots
    ENV["GKSwstype"]="nul"

"nul"

In [5]:
en = TS_env(init_env(Int(100), Float32(10.0), Float32(0.2), Float32(1.0), Float32(0.7), Float32(0.5))...)

ag = agtQ(init_nQ(en, Int(512), Float32(0.95), Float32(3.0))...)

    #二次の高周波展開で初期値を代入
ag.HF_TL[en.t_size,:] = init_HF(en)
ag.K_TL[en.t_size,:] = zeros(Float32, en.HS_size^2)
    #-MtoV(en.V_t, en)/en.Ω

    #model = Chain(Dense(ag.in_size, ag.n_dense, tanh), Dense(ag.n_dense, ag.n_dense, tanh), Dense(ag.n_dense, ag.n_dense, tanh), Dense(ag.n_dense, ag.out_size))
model = Chain(Dense(ag.in_size, ag.n_dense, tanh), Dense(ag.n_dense, ag.n_dense, tanh), Dense(ag.n_dense, ag.out_size))
model_g = fmap(cu, model)
    #model = Chain(Dense(zeros(Float32, ag.n_dense, ag.in_size), zeros(Float32, ag.n_dense), tanh), Dense(zeros(Float32, ag.n_dense, ag.n_dense), zeros(Float32, ag.n_dense), tanh), Dense(zeros(Float32, ag.out_size, ag.n_dense), zeros(Float32, ag.out_size)))
opt = ADAM()

Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [13]:
HF_it = zeros(Float32, en.HS_size^2) 

for t_step in 1:en.t_size
        if(t_step==1)
                tt=en.t_size
        else
                tt=t_step-1
        end
        p = [en.Ω, en.ξ*sin(2pi*t_step/en.t_size), en.Jz, en.Jx, en.hz]
        x = vcat([p, ag.K_TL[tt,:], ag.Kp_TL[tt,:]]...)
        xg = gpu(x)
        ag.Kp_TL[t_step,:] = Array(model_g(xg))
        ag.K_TL[t_step,:], ag.HF_TL[t_step,:] = micro_motion(ag.Kp_TL[t_step,:], ag.K_TL[tt,:],en,t_step)
        #ag.K_TL[t,:] += Kp
        HF_it += ag.HF_TL[t_step,:]/en.t_size
end
println("HF_calc Finish!")

HF_calc Finish!


In [8]:
HF_it = zeros(Float32, en.HS_size^2) 

for t_step in 1:en.t_size
        if(t_step==1)
                tt=en.t_size
        else
                tt=t_step-1
        end
        p = [en.Ω, en.ξ*sin(2pi*t_step/en.t_size), en.Jz, en.Jx, en.hz]
        x = vcat([p, ag.K_TL[tt,:], ag.Kp_TL[tt,:]]...)
        #xg = gpu(x)
        ag.Kp_TL[t_step,:] = model(x)
        ag.K_TL[t_step,:], ag.HF_TL[t_step,:] = micro_motion(ag.Kp_TL[t_step,:], ag.K_TL[tt,:],en,t_step)
        #ag.K_TL[t,:] += Kp
        HF_it += ag.HF_TL[t_step,:]/en.t_size
end
println("HF_calc Finish!")

HF_calc Finish!


In [19]:
grads = Flux.gradient(Flux.params(model_g)) do
        loss_calc_hyb(model_g, en, ag, HF_it)
        #loss_calc0(model, en, ag, t_step, HF_it)
        #loss_t(model, en, ag, t_step, it)
        #loss_t!(model, en, ag, t_step, it)
end
Flux.Optimise.update!(opt, Flux.params(model_g), grads)
println("Finish!")

GPUCompiler.KernelError: GPU compilation of kernel #broadcast_kernel#17(CUDA.CuKernelContext, CuDeviceMatrix{Float32, 1}, Base.Broadcast.Broadcasted{CUDA.CuArrayStyle{2}, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}}, typeof(*), Tuple{Base.Broadcast.Extruded{Vector{Float32}, Tuple{Bool}, Tuple{Int64}}, Base.Broadcast.Extruded{Adjoint{Float32, CuDeviceVector{Float32, 1}}, Tuple{Bool, Bool}, Tuple{Int64, Int64}}}}, Int64) failed
KernelError: passing and using non-bitstype argument

Argument 4 to your kernel function is of type Base.Broadcast.Broadcasted{CUDA.CuArrayStyle{2}, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}}, typeof(*), Tuple{Base.Broadcast.Extruded{Vector{Float32}, Tuple{Bool}, Tuple{Int64}}, Base.Broadcast.Extruded{Adjoint{Float32, CuDeviceVector{Float32, 1}}, Tuple{Bool, Bool}, Tuple{Int64, Int64}}}}, which is not isbits:
  .args is of type Tuple{Base.Broadcast.Extruded{Vector{Float32}, Tuple{Bool}, Tuple{Int64}}, Base.Broadcast.Extruded{Adjoint{Float32, CuDeviceVector{Float32